## Nest light curves

Nest source and forced source in the DIA object catalog.

In [1]:
import lsdb
import os

from dask.distributed import Client
from hats_import import pipeline_with_client
from hats_import.margin_cache.margin_cache_arguments import MarginCacheArguments
from pathlib import Path

In [2]:
# Path to the RAW parquet data
PPDB_DIR = Path("/sdf/scratch/rubin/ppdb/data/lsstcam")

# Paths to the target OUTPUT directories
TMP_DIR = Path(os.environ["OUTPUT_DIR"]) / "tmp"

In [3]:
client = Client(n_workers=16, threads_per_worker=1, local_directory=TMP_DIR)

Prepare a margin cache for the source catalogs so that we can nest without losing data: 

In [ ]:
def load_sources_with_margin(dataset_type):
    margin_threshold = 5
    main_catalog_path = TMP_DIR / dataset_type
    margin_name = f"{dataset_type}_{margin_threshold}arcs"
    try:
        args = MarginCacheArguments(
            input_catalog_path=main_catalog_path,
            output_path=TMP_DIR,
            margin_threshold=margin_threshold,
            output_artifact_name=margin_name,
            simple_progress_bar=True,
            resume=False,
        )
        pipeline_with_client(args, client)
    except ValueError as e:
        print(dataset_type, e)
        catalog = lsdb.read_hats(main_catalog_path)
    else:
        catalog = lsdb.read_hats(main_catalog_path, margin_cache=TMP_DIR / margin_name)
    return catalog

In [5]:
dia_source = load_sources_with_margin("dia_source")
dia_forced_source = load_sources_with_margin("dia_forced_source")

Binning   :   0%|                                                                                                                                                                                                                                                                                                                        | 0/1 [00:00<?, ?it/s]


dia_source Margin cache contains no rows. Increase margin size and re-run.


Binning   :   0%|                                                                                                                                                                                                                                                                                                                        | 0/1 [00:00<?, ?it/s]


dia_forced_source Margin cache contains no rows. Increase margin size and re-run.


Nest and use `map_partitions` to sort the sources chronologically:

In [6]:
dia_object = lsdb.read_hats(TMP_DIR / "dia_object")

In [7]:
def sort_nested_sources(df, source_cols):
    mjd_col = "midpointMjdTai"
    for source_col in source_cols:
        flat_sources = df[source_col].nest.to_flat()
        df = df.drop(columns=[source_col])
        df = df.add_nested(
            flat_sources.sort_values([flat_sources.index.name, mjd_col]), source_col
        )
    return df

In [8]:
dia_object_nested = (
    dia_object.join_nested(
        dia_source,
        left_on="diaObjectId",
        right_on="diaObjectId",
        nested_column_name="diaSource",
    )
    .join_nested(
        dia_forced_source,
        left_on="diaObjectId",
        right_on="diaObjectId",
        nested_column_name="diaForcedSource",
    )
    .map_partitions(
        lambda x: sort_nested_sources(x, source_cols=["diaSource", "diaForcedSource"])
    )
)
dia_object_nested

/sdf/home/s/stavar/.local/lib/python3.12/site-packages/lsdb/dask/join_catalog_data.py:335: RuntimeWarning: Right catalog does not have a margin cache. Results may be incomplete and/or inaccurate.
  warnings.warn(
/sdf/home/s/stavar/.local/lib/python3.12/site-packages/lsdb/dask/join_catalog_data.py:335: RuntimeWarning: Right catalog does not have a margin cache. Results may be incomplete and/or inaccurate.
  warnings.warn(


,diaObjectId,validityStart,dec,decErr,g_fpFluxMean,g_fpFluxMeanErr,g_fpFluxSigma,g_psfFluxChi2,g_psfFluxErrMean,g_psfFluxLinearIntercept,g_psfFluxLinearSlope,g_psfFluxMAD,g_psfFluxMax,g_psfFluxMaxSlope,g_psfFluxMean,g_psfFluxMeanErr,g_psfFluxMin,g_psfFluxNdata,g_psfFluxPercentile05,g_psfFluxPercentile25,g_psfFluxPercentile50,g_psfFluxPercentile75,g_psfFluxPercentile95,g_psfFluxSigma,g_psfFluxSkew,g_psfFluxStetsonJ,g_scienceFluxMean,g_scienceFluxMeanErr,g_scienceFluxSigma,i_fpFluxMean,i_fpFluxMeanErr,i_fpFluxSigma,i_psfFluxChi2,i_psfFluxErrMean,i_psfFluxLinearIntercept,i_psfFluxLinearSlope,i_psfFluxMAD,i_psfFluxMax,i_psfFluxMaxSlope,i_psfFluxMean,i_psfFluxMeanErr,i_psfFluxMin,i_psfFluxNdata,i_psfFluxPercentile05,i_psfFluxPercentile25,i_psfFluxPercentile50,i_psfFluxPercentile75,i_psfFluxPercentile95,i_psfFluxSigma,i_psfFluxSkew,i_psfFluxStetsonJ,i_scienceFluxMean,i_scienceFluxMeanErr,i_scienceFluxSigma,lastNonForcedSource,nDiaSources,nearbyExtObj1,nearbyExtObj1Sep,nearbyExtObj2,nearbyExtObj2Sep,nearbyExtObj3,nearbyExtObj3Sep,nearbyLowzGal,nearbyLowzGalSep,nearbyObj1,nearbyObj1Dist,nearbyObj1LnP,nearbyObj2,nearbyObj2Dist,nearbyObj2LnP,nearbyObj3,nearbyObj3Dist,nearbyObj3LnP,parallax,parallaxErr,pmDec,pmDecErr,pmDec_parallax_Cov,pmParallaxChi2,pmParallaxLnL,pmParallaxNdata,pmRa,pmRaErr,pmRa_parallax_Cov,pmRa_pmDec_Cov,r_fpFluxMean,r_fpFluxMeanErr,r_fpFluxSigma,r_psfFluxChi2,r_psfFluxErrMean,r_psfFluxLinearIntercept,r_psfFluxLinearSlope,r_psfFluxMAD,r_psfFluxMax,r_psfFluxMaxSlope,r_psfFluxMean,r_psfFluxMeanErr,r_psfFluxMin,r_psfFluxNdata,r_psfFluxPercentile05,r_psfFluxPercentile25,r_psfFluxPercentile50,r_psfFluxPercentile75,r_psfFluxPercentile95,r_psfFluxSigma,r_psfFluxSkew,r_psfFluxStetsonJ,r_scienceFluxMean,r_scienceFluxMeanErr,r_scienceFluxSigma,ra,raErr,ra_dec_Cov,radecMjdTai,u_fpFluxMean,u_fpFluxMeanErr,u_fpFluxSigma,u_psfFluxChi2,u_psfFluxErrMean,u_psfFluxLinearIntercept,u_psfFluxLinearSlope,u_psfFluxMAD,u_psfFluxMax,u_psfFluxMaxSlope,u_psfFluxMean,u_psfFluxMeanErr,u_psfFluxMin,u_psfFluxNdata,u_psfFluxPercentile05,u_psfFluxPercentile25,u_psfFluxPercentile50,u_psfFluxPercentile75,u_psfFluxPercentile95,u_psfFluxSigma,u_psfFluxSkew,u_psfFluxStetsonJ,u_scienceFluxMean,u_scienceFluxMeanErr,u_scienceFluxSigma,validityEnd,y_fpFluxMean,y_fpFluxMeanErr,y_fpFluxSigma,y_psfFluxChi2,y_psfFluxErrMean,y_psfFluxLinearIntercept,y_psfFluxLinearSlope,y_psfFluxMAD,y_psfFluxMax,y_psfFluxMaxSlope,y_psfFluxMean,y_psfFluxMeanErr,y_psfFluxMin,y_psfFluxNdata,y_psfFluxPercentile05,y_psfFluxPercentile25,y_psfFluxPercentile50,y_psfFluxPercentile75,y_psfFluxPercentile95,y_psfFluxSigma,y_psfFluxSkew,y_psfFluxStetsonJ,y_scienceFluxMean,y_scienceFluxMeanErr,y_scienceFluxSigma,z_fpFluxMean,z_fpFluxMeanErr,z_fpFluxSigma,z_psfFluxChi2,z_psfFluxErrMean,z_psfFluxLinearIntercept,z_psfFluxLinearSlope,z_psfFluxMAD,z_psfFluxMax,z_psfFluxMaxSlope,z_psfFluxMean,z_psfFluxMeanErr,z_psfFluxMin,z_psfFluxNdata,z_psfFluxPercentile05,z_psfFluxPercentile25,z_psfFluxPercentile50,z_psfFluxPercentile75,z_psfFluxPercentile95,z_psfFluxSigma,z_psfFluxSkew,z_psfFluxStetsonJ,z_scienceFluxMean,z_scienceFluxMeanErr,z_scienceFluxSigma,u_scienceMagMean,u_scienceMagMeanErr,g_scienceMagMean,g_scienceMagMeanErr,r_scienceMagMean,r_scienceMagMeanErr,i_scienceMagMean,i_scienceMagMeanErr,z_scienceMagMean,z_scienceMagMeanErr,y_scienceMagMean,y_scienceMagMeanErr,diaSource,diaForcedSource
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Order: 0, Pixel: 6",int64[pyarrow],"timestamp[ms, tz=UTC][pyarrow]",double[pyarrow],double[pyarrow],float[pyarrow],float[pyarrow],float[pyarrow],float[pyarrow],float[pyarrow],float[pyarrow],float[pyarrow],float[pyarrow],float[pyarrow],float[pyarrow],float[pyarrow],float[pyarrow],float[pyarrow],int32[pyarrow],float[pyarrow],float[pyarrow],float[pyarrow],float[pyarrow],float[pyarrow],float[pyarrow],f

Save the new data catalog to disk:

In [9]:
dia_object_nested.to_hats(
    TMP_DIR / "new_dia_object_lc", catalog_name="dia_object_lc", as_collection=False
)

Free up some space from intermediate catalogs that are no longer needed:

In [10]:
%rm -rf $TMP_DIR/dia_object*
%rm -rf $TMP_DIR/dia_source*
%rm -rf $TMP_DIR/dia_forced_source*

In [11]:
client.close()